# Neurosynth ML Decoding
The goal is to do reverse inference **(Brain Activation Pattern→Terms)** using machine learning. 

Ideally, a trained machine learner, **F**, should output terms strongly associated with a brain activation pattern. 

For example, given the activation of the left amygdala @ [x,y,z]=[-22,0,-20], F([-22,0,-20]) will output/indicate **"amygdala", "fear", etc**.

## 0. Loading Neurosynth Dataset

In [105]:
from nimare import dataset as nimare_dataset
import os

PICKLE_PATH = "neurosynth_dataset.pkl.gz"
dset = nimare_dataset.Dataset.load(PICKLE_PATH)
print("n_studies =", len(dset.ids))
print("shapes: coords", dset.coordinates.shape, "meta", dset.metadata.shape, "ann", dset.annotations.shape)

n_studies = 14371
shapes: coords (507891, 7) meta (14371, 7) ann (14371, 3231)


In [106]:
# Not needed for this example
dset.metadata

,id,study_id,contrast_id,authors,journal,year,title
0,10022492-1,10022492,1,"Callicott JH, Mattay VS, Bertolino A, Finn K, ...","Cerebral cortex (New York, N.Y. : 1991)",1999,Physiological characteristics of capacity cons...
1,10022494-1,10022494,1,"Toni I, Schluter ND, Josephs O, Friston K, Pas...","Cerebral cortex (New York, N.Y. : 1991)",1999,"Signal-, set- and movement-related activity in..."
2,10022496-1,10022496,1,"Lockwood AH, Salvi RJ, Coad ML, Arnold SA, Wac...","Cerebral cortex (New York, N.Y. : 1991)",1999,The functional anatomy of the normal human aud...
3,10051677-1,10051677,1,"Denton D, Shade R, Zamarippa F, Egan G, Blair-...",Proceedings of the National Academy of Science...,1999,Correlation of regional cerebral blood flow an...
4,10191322-1,10191322,1,"Chee MW, Tan EW, Thiel T",The Journal of neuroscience : the official jou...,1999,Mandarin and English single word processing st...
...,...,...,...,...,...,...,...
14366,9819274-1,9819274,1,"Lobel E, Kleine JF, Bihan DL, Leroy-Willig A, ...",Journal of neurophysiology,1998,Functional MRI of galvanic vestibular stimulat...
14367,9838166-1,9838166,1,"Wildgruber D, Kischka U, Ackermann H, Klose U,...",Brain research. Cognitive brain research,1999,Dynamic pattern of brain activation during seq...
14368,9862924-1,9862924,1,"Porro CA, Cettolo V, Francescato MP, Baraldi P",Journal of neurophysiology,1998,Temporal and intensity coding of pain in human...
14369,9886448-1,9886448,1,"Van Der Werf YD, Weerts JG, Jolles J, Witter M...","Journal of neurology, neurosurgery, and psychi...",1999,Neuropsychological correlates of a right unila...


In [107]:
# 查看 metadata 的基本資訊
print("=== dset.metadata 基本資訊 ===")
print(f"Shape: {dset.metadata.shape}")
print(f"Columns: {list(dset.metadata.columns)}")
print("\n前5筆資料:")
print(dset.metadata.head())
print("\n資料型態:")
print(dset.metadata.dtypes)

=== dset.metadata 基本資訊 ===
Shape: (14371, 7)
Columns: ['id', 'study_id', 'contrast_id', 'authors', 'journal', 'year', 'title']

前5筆資料:
           id  study_id contrast_id  \
0  10022492-1  10022492           1   
1  10022494-1  10022494           1   
2  10022496-1  10022496           1   
3  10051677-1  10051677           1   
4  10191322-1  10191322           1   

                                             authors  \
0  Callicott JH, Mattay VS, Bertolino A, Finn K, ...   
1  Toni I, Schluter ND, Josephs O, Friston K, Pas...   
2  Lockwood AH, Salvi RJ, Coad ML, Arnold SA, Wac...   
3  Denton D, Shade R, Zamarippa F, Egan G, Blair-...   
4                           Chee MW, Tan EW, Thiel T   

                                             journal  year  \
0            Cerebral cortex (New York, N.Y. : 1991)  1999   
1            Cerebral cortex (New York, N.Y. : 1991)  1999   
2            Cerebral cortex (New York, N.Y. : 1991)  1999   
3  Proceedings of the National Academy of Sci

## 1. Brain Activation Clusters
Apply an unsupervised learning algorithm to cluster 507,891 coordinates into M clusters. The optimal value of M can be determined using the Elbow method or other appropriate techniques.

Save the following results:

[1] The cluster assignment (category membership) for each of the 507,891 coordinates.

[2] The coordinates of each cluster center (prototype).

In [108]:
dset.coordinates

,id,study_id,contrast_id,x,y,z,space
1483,10022492-1,10022492,1,36.0,-58.0,52.0,mni152_2mm
1499,10022492-1,10022492,1,48.0,24.0,20.0,mni152_2mm
1498,10022492-1,10022492,1,-42.0,26.0,20.0,mni152_2mm
1497,10022492-1,10022492,1,-36.0,30.0,16.0,mni152_2mm
1496,10022492-1,10022492,1,-30.0,32.0,0.0,mni152_2mm
...,...,...,...,...,...,...,...
1479,9990082-1,9990082,1,42.0,-54.0,-21.0,mni152_2mm
1480,9990082-1,9990082,1,-36.0,-87.0,-6.0,mni152_2mm
1481,9990082-1,9990082,1,30.0,-81.0,-15.0,mni152_2mm
1467,9990082-1,9990082,1,-18.0,-60.0,54.0,mni152_2mm


In [109]:
# Write your code here:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

coords = dset.coordinates[["x", "y", "z"]].values
inertias = []
K_range = range(2, 50)  # 可以調整，比如 2~50

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(coords)
    inertias.append(kmeans.inertia_)
    print(f"k={k}, inertia={kmeans.inertia_}")

plt.plot(K_range, inertias, marker="o")
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia (SSE)")
plt.title("Elbow Method for Optimal k")
plt.show()

In [110]:
M = 20  # 選擇最佳 k 值
kmeans = KMeans(n_clusters=M, random_state=42, n_init=10)
dset.coordinates["cluster"] = kmeans.fit_predict(coords)
print(dset.coordinates.head())

# 把 cluster assignment 加回 DataFrame
dset.coordinates["cluster"] = kmeans.labels_

# 叢集中心座標
print(f"Cluster centers shape: {kmeans.cluster_centers_.shape}")
centers = pd.DataFrame(kmeans.cluster_centers_, columns=["x", "y", "z"])
print("Cluster centers:")
print(centers)

# 存檔
dset.coordinates.to_csv("coordinates_with_clusters.csv", index=False)
centers.to_csv("cluster_centers.csv", index=False)

              id  study_id contrast_id     x     y     z       space  cluster
1483  10022492-1  10022492           1  36.0 -58.0  52.0  mni152_2mm       19
1499  10022492-1  10022492           1  48.0  24.0  20.0  mni152_2mm       22
1498  10022492-1  10022492           1 -42.0  26.0  20.0  mni152_2mm       48
1497  10022492-1  10022492           1 -36.0  30.0  16.0  mni152_2mm       48
1496  10022492-1  10022492           1 -30.0  32.0   0.0  mni152_2mm       31
Cluster centers shape: (50, 3)
Cluster centers:
            x          y          z
0   55.186799 -23.112148  29.511843
1  -46.951099   5.092401  33.271187
2  -21.802587 -80.733644  25.852184
3    9.935642   9.124188   1.733936
4   21.041838 -85.775296  -2.750540
5    0.339444 -22.852028  37.168154
6   26.199109 -37.029669  -9.079335
7  -28.206689  -4.827793 -22.067561
8  -34.262573 -44.005173 -16.423875
9    0.924333 -32.097315  64.073030
10  34.981251  51.827188   3.032748
11 -33.792902  51.208324   6.061066
12  49.705740   

## 2. Characterize Each Study as an M-Dimensional Vector

Use the Bag-of-Visual-Words (BoVW) approach to represent each study as an M-dimensional vector.

In this context, a bag corresponds to a study, and a visual word corresponds to a cluster. The resulting M-dimensional vector is a histogram that records the frequency of each activation cluster within the study.

In [111]:
# Step 2: Bag-of-Visual-Words (BoVW) 表示法
# 取得所有 study_id 與 cluster assignment
coords = dset.coordinates[["study_id", "cluster"]]
M = coords["cluster"].nunique()  # cluster 數量

# 計算每個 study 在每個 cluster 的出現次數
bovw = coords.groupby(["study_id", "cluster"]).size().unstack(fill_value=0)
bovw = bovw.sort_index(axis=1)  # cluster 由小到大排序

print(f"BoVW shape: {bovw.shape}")
print("前5個研究的 BoVW 向量：")
print(bovw.head())

# 若要用於後續 ML，可將 bovw 轉為 numpy array 或 DataFrame
# bovw.values  # numpy array
# bovw         # DataFrame

BoVW shape: (14371, 50)
前5個研究的 BoVW 向量：
cluster   0   1   2   3   4   5   6   7   8   9   ...  40  41  42  43  44  45  \
study_id                                          ...                           
10022492   0   0   0   0   0   0   0   0   0   0  ...   2   0   0   0   0   0   
10022494   1   0   2   0   3   0   0   0   0   2  ...   1   0   0   1   0   2   
10022496   0   0   0   0   0   0   0   0   2   0  ...   0   0   0   0   0  23   
10051677   0   1   0   0   0   1   3   0   1   0  ...   0   0   2   1   2   0   
10191322   0  16   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

cluster   46  47  48  49  
study_id                  
10022492   0   0   2   0  
10022494   0   0   1   0  
10022496   0   0   0   0  
10051677   0   1   1   2  
10191322   0   0  26   0  

[5 rows x 50 columns]


## 3. First Approach: kNN Decoding

Convert the left amygdala activation at [x,y,z]=[−22,0,−20] into its M-dimensional representation.

Identify its k nearest neighbors, and compute the average of their associated term vectors from dset.annotations. You can also use "KNeighborsRegressor" in scikit-learn. Report the terms with the highest averaged values (i.e., the winning terms).

In [112]:
dset.annotations

,id,study_id,contrast_id,terms_abstract_tfidf__001,terms_abstract_tfidf__01,terms_abstract_tfidf__05,terms_abstract_tfidf__10,terms_abstract_tfidf__100,terms_abstract_tfidf__11,terms_abstract_tfidf__12,...,terms_abstract_tfidf__yield,terms_abstract_tfidf__yielded,terms_abstract_tfidf__young,terms_abstract_tfidf__young adults,terms_abstract_tfidf__young healthy,terms_abstract_tfidf__young older,terms_abstract_tfidf__younger,terms_abstract_tfidf__younger adults,terms_abstract_tfidf__youth,terms_abstract_tfidf__zone
0,10022492-1,10022492,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10022494-1,10022494,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10022496-1,10022496,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10051677-1,10051677,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10191322-1,10191322,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.079103,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14366,9819274-1,9819274,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.109049,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14367,9838166-1,9838166,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.088527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14368,9862924-1,9862924,1,0.055394,0.0,0.0,0.0,0.055598,0.04457,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14369,9886448-1,9886448,1,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
# 查看 annotations 的基本資訊
print("=== dset.annotations 基本資訊 ===")
print(f"Shape: {dset.annotations.shape}")
print(f"Columns (前10個): {list(dset.annotations.columns[:10])}")
print(f"總共有 {len(dset.annotations.columns)} 個詞彙/標註")
print("\n前5筆資料 (前10個特徵):")
print(dset.annotations.iloc[:5, :10])
print("\n資料型態:")
print(dset.annotations.dtypes.value_counts())

=== dset.annotations 基本資訊 ===
Shape: (14371, 3231)
Columns (前10個): ['id', 'study_id', 'contrast_id', 'terms_abstract_tfidf__001', 'terms_abstract_tfidf__01', 'terms_abstract_tfidf__05', 'terms_abstract_tfidf__10', 'terms_abstract_tfidf__100', 'terms_abstract_tfidf__11', 'terms_abstract_tfidf__12']
總共有 3231 個詞彙/標註

前5筆資料 (前10個特徵):
           id  study_id contrast_id  terms_abstract_tfidf__001  \
0  10022492-1  10022492           1                        0.0   
1  10022494-1  10022494           1                        0.0   
2  10022496-1  10022496           1                        0.0   
3  10051677-1  10051677           1                        0.0   
4  10191322-1  10191322           1                        0.0   

   terms_abstract_tfidf__01  terms_abstract_tfidf__05  \
0                       0.0                       0.0   
1                       0.0                       0.0   
2                       0.0                       0.0   
3                       0.0                

In [103]:
# 查看一些具體的詞彙範例
print("=== 詞彙範例 ===")
# 取得所有詞彙欄位（排除 id, study_id, contrast_id）
term_columns = [col for col in dset.annotations.columns if col.startswith('terms_')]
print(f"詞彙欄位總數: {len(term_columns)}")

# 顯示一些詞彙名稱（移除前綴）
sample_terms = [col.replace('terms_abstract_tfidf__', '') for col in term_columns[:20]]
print(f"前20個詞彙: {sample_terms}")

# 查看某個研究的非零詞彙
print("\n=== 第一個研究的非零詞彙 (前10個) ===")
first_study = dset.annotations.iloc[0, 3:]  # 排除前3個欄位
non_zero_terms = first_study[first_study > 0]
print(f"非零詞彙數量: {len(non_zero_terms)}")
if len(non_zero_terms) > 0:
    top_terms = non_zero_terms.sort_values(ascending=False).head(10)
    for term, value in top_terms.items():
        clean_term = term.replace('terms_abstract_tfidf__', '')
        print(f"  {clean_term}: {value:.4f}")

=== 詞彙範例 ===
詞彙欄位總數: 3228
前20個詞彙: ['001', '01', '05', '10', '100', '11', '12', '12 healthy', '13', '14', '14 healthy', '15', '15 healthy', '16', '16 healthy', '17', '18', '18 healthy', '19', '20']

=== 第一個研究的非零詞彙 (前10個) ===
非零詞彙數量: 77
  capacity: 0.6235
  working memory: 0.3204
  working: 0.3176
  constrained: 0.2381
  memory: 0.2379
  dlpfc: 0.1681
  physiological: 0.1244
  characteristics: 0.1210
  load: 0.1195
  increasing: 0.1039


In [113]:
# Step 3: kNN Decoding
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# 設定 M = 20 (cluster 數量)
M = 20

# 1. 將左側杏仁核座標 [-22, 0, -20] 轉換為 M 維表示
target_coord = np.array([[-22, 0, -20]])
target_cluster = kmeans.predict(target_coord)[0]
print(f"左側杏仁核座標 [-22, 0, -20] 被分配到 cluster {target_cluster}")

# 創建目標點的 M 維 BoVW 向量 (只有對應 cluster 為 1，其他為 0)
target_bovw = np.zeros(M)
target_bovw[target_cluster] = 1
print(f"目標點的 M 維向量: {target_bovw}")

# 2. 準備訓練資料
# X: 每個研究的 M 維 BoVW 向量
# y: 每個研究對應的詞彙向量 (annotations)

# 確保 bovw 與 annotations 的 study_id 對應
annotations_terms = dset.annotations.drop(['id', 'contrast_id'], axis=1)
merged_data = annotations_terms.merge(bovw, left_on='study_id', right_index=True, how='inner')

# 分離特徵 (BoVW) 和目標 (terms)
X = merged_data.iloc[:, -M:].values  # 最後 M 欄是 BoVW
y = merged_data.iloc[:, 1:-M].values  # 中間部分是 terms (排除 study_id)

print(f"訓練資料形狀: X={X.shape}, y={y.shape}")

# 3. 訓練 kNN 模型
N = X.shape[0]  # 訓練資料數量
k = int(np.sqrt(N))  # k 值設為根號 N
print(f"訓練資料數量 N = {N}, k = sqrt(N) = {k}")
knn = KNeighborsRegressor(n_neighbors=k)
knn.fit(X, y)

# 4. 預測目標點的詞彙向量
predicted_terms = knn.predict([target_bovw])[0]

# 5. 找出最高分的詞彙
term_columns = [col for col in dset.annotations.columns if col.startswith('terms_')]
clean_term_names = [col.replace('terms_abstract_tfidf__', '') for col in term_columns]

# 創建詞彙-分數對應
term_scores = list(zip(clean_term_names, predicted_terms))
term_scores.sort(key=lambda x: x[1], reverse=True)

print(f"\n=== kNN 解碼結果 (k={k}) ===")
print("前10個最相關的詞彙:")
for i, (term, score) in enumerate(term_scores[:10]):
    print(f"{i+1:2d}. {term:20s}: {score:.4f}")

左側杏仁核座標 [-22, 0, -20] 被分配到 cluster 7
目標點的 M 維向量: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
訓練資料形狀: X=(14371, 50), y=(14371, 3228)
訓練資料數量 N = 14371, k = sqrt(N) = 119

=== kNN 解碼結果 (k=119) ===
前10個最相關的詞彙:
 1. schizophrenia       : 0.0363
 2. amygdala            : 0.0348
 3. matter              : 0.0296
 4. white matter        : 0.0241
 5. white               : 0.0235
 6. cortical            : 0.0227
 7. state               : 0.0225
 8. social              : 0.0223
 9. healthy             : 0.0207
10. group               : 0.0195
訓練資料形狀: X=(14371, 50), y=(14371, 3228)
訓練資料數量 N = 14371, k = sqrt(N) = 119

=== kNN 解碼結果 (k=119) ===
前10個最相關的詞彙:
 1. schizophrenia       : 0.0363
 2. amygdala            : 0.0348
 3. matter              : 0.0296
 4. white matter        : 0.0241
 5. white               : 0.0235
 6. cortical            : 0.0227
 7. state               : 0.0225
 8. social

## 4. Second Approach: Decoding with Data-Compressing Models

Unlike k-NN, which does not compress training data, 
this approach employs models that learn a compressed representation. 
Two options to consider are MultiOutputRegressor and MLPRegressor.

These models can learn to map an M-dimensional study vector to its corresponding term vector. 
Train and fine-tune your chosen model, then evaluate it on the M-dimensional representation of the left amygdala activation at 
[x,y,z]=[−22,0,−20].

Report the terms with the highest predicted values (i.e., the winning terms).

In [126]:
# Step 4: MultiOutputRegressor Decoding
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 設定 M = 20 (cluster 數量)
M = 20

# 1. 使用與 kNN 相同的資料準備
print("=== 資料準備 ===")

# 確保 bovw 與 annotations 的 study_id 對應
annotations_terms = dset.annotations.drop(['id', 'contrast_id'], axis=1)
merged_data = annotations_terms.merge(bovw, left_on='study_id', right_index=True, how='inner')

# 分離特徵 (BoVW) 和目標 (terms)
X = merged_data.iloc[:, -M:].values  # 最後 M 欄是 BoVW
y = merged_data.iloc[:, 1:-M].values  # 中間部分是 terms (排除 study_id)

print(f"訓練資料形狀: X={X.shape}, y={y.shape}")

# 2. 分割訓練/測試資料
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"訓練集: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"測試集: X_test={X_test.shape}, y_test={y_test.shape}")

# 3. 訓練 MultiOutputRegressor 模型
print("\n=== 模型訓練 ===")

# 選項1: MultiOutputRegressor + RandomForestRegressor
model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1))
model_rf.fit(X_train, y_train)

# 選項2: MultiOutputRegressor + Ridge Regression
model_ridge = MultiOutputRegressor(Ridge(alpha=1.0))
model_ridge.fit(X_train, y_train)

print("模型訓練完成!")

# 4. 模型評估
print("\n=== 模型評估 ===")
y_pred_rf = model_rf.predict(X_test)
y_pred_ridge = model_ridge.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

print(f"RandomForest MSE: {mse_rf:.6f}")
print(f"Ridge MSE: {mse_ridge:.6f}")

# 選擇較好的模型
if mse_rf < mse_ridge:
    best_model = model_rf
    best_model_name = "RandomForest"
else:
    best_model = model_ridge
    best_model_name = "Ridge"

print(f"最佳模型: {best_model_name}")

# 5. 預測左側杏仁核座標的詞彙
print("\n=== 左側杏仁核解碼 ===")

# 創建目標點的 M 維 BoVW 向量
target_coord = np.array([[-22, 0, -20]])
target_cluster = kmeans.predict(target_coord)[0]
target_bovw = np.zeros(M)
target_bovw[target_cluster] = 1

print(f"左側杏仁核座標 [-22, 0, -20] 被分配到 cluster {target_cluster}")
print(f"目標點的 M 維向量: {target_bovw}")

# 預測詞彙向量
predicted_terms = best_model.predict([target_bovw])[0]

# 6. 找出最高分的詞彙
term_columns = [col for col in dset.annotations.columns if col.startswith('terms_')]
clean_term_names = [col.replace('terms_abstract_tfidf__', '') for col in term_columns]

# 創建詞彙-分數對應
term_scores = list(zip(clean_term_names, predicted_terms))
term_scores.sort(key=lambda x: x[1], reverse=True)

print(f"\n=== {best_model_name} 解碼結果 ===")
print("前15個最相關的詞彙:")
for i, (term, score) in enumerate(term_scores[:15]):
    print(f"{i+1:2d}. {term:25s}: {score:.6f}")

# 7. 比較正負值
positive_terms = [(term, score) for term, score in term_scores if score > 0]
print(f"\n正值詞彙數量: {len(positive_terms)}")
if len(positive_terms) > 0:
    print("前5個正值詞彙:")
    for i, (term, score) in enumerate(positive_terms[:5]):
        print(f"  {i+1}. {term:25s}: {score:.6f}")

=== 資料準備 ===
訓練資料形狀: X=(14371, 20), y=(14371, 3228)
訓練集: X_train=(11496, 20), y_train=(11496, 3228)
測試集: X_test=(2875, 20), y_test=(2875, 3228)

=== 模型訓練 ===
模型訓練完成!

=== 模型評估 ===
RandomForest MSE: 0.000343
Ridge MSE: 0.000299
最佳模型: Ridge

=== 左側杏仁核解碼 ===
左側杏仁核座標 [-22, 0, -20] 被分配到 cluster 5
目標點的 M 維向量: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

=== Ridge 解碼結果 ===
前15個最相關的詞彙:
 1. task                     : 0.025221
 2. connectivity             : 0.021833
 3. memory                   : 0.021261
 4. visual                   : 0.019460
 5. gyrus                    : 0.019281
 6. temporal                 : 0.019277
 7. response                 : 0.017937
 8. network                  : 0.017714
 9. frontal                  : 0.017696
10. motor                    : 0.017451
11. control                  : 0.017352
12. cognitive                : 0.017026
13. amygdala                 : 0.016465
14. using                    : 0.016404
15. prefrontal               : 0.016145

